In [12]:
# univariate cnn lstm example
from numpy import array

import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import TimeSeriesSplit

import numpy
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pandas as pd
from pandas import read_csv
from math import sqrt


# split a univariate sequence into samples

def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

def train_test (n_steps_in, n_steps_out, propTrainTest, dataset):
    
    n_steps_in, n_steps_out = n_steps_in, n_steps_out
    # split train test
    X, y = split_sequence(dataset, n_steps_in, n_steps_out)
    
    train_size = int(len(X) * propTrainTest)
    test_size = len(dataset) - train_size
    train_X, test_X = X[0:train_size,:], X[train_size:len(X),:]
    train_Y, test_Y = y[0:train_size,:], y[train_size:len(y),:]
    
    return (train_X, test_X, train_Y, test_Y)

# fixem random seed
numpy.random.seed(7)

dadesSau = read_csv('dadesBaells.csv', sep=';',header=0, index_col=0)
dataframe = pd.DataFrame(dadesSau.loc[dadesSau.index >= '2000-01-01']['Volum'])
dataset = dataframe.values
dataset = dataset.astype('float32')

# Normalitzem

scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)
dataset = [item for sublist in dataset for item in sublist]

n_steps_in = 20
n_steps_out = 15
propTrainTest = 0.8
train_X, test_X, train_Y, test_Y = train_test(n_steps_in, n_steps_out, propTrainTest, dataset)


In [2]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = np.arange(10,501,20).tolist()
# Number of features to consider at every split
max_features = [1,5,6,7,10,12,15,20]
# Maximum number of levels in tree
max_depth = np.arange(1,121,5).tolist()
# Minimum number of samples required to split a node
min_samples_split = np.arange(2,50,3)
# Minimum number of samples required at each leaf node
min_samples_leaf = np.arange(1,41,5).tolist()


In [3]:
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              }

In [4]:
rf = RandomForestRegressor()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 500, cv=TimeSeriesSplit(n_splits=5).get_n_splits([train_X,train_Y]), scoring='neg_mean_squared_error', random_state = 125, n_jobs=-1, verbose=0)

rf_random.fit(train_X, train_Y)

RandomizedSearchCV(cv=5, error_score='raise',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid=True, n_iter=500, n_jobs=-1,
          param_distributions={'n_estimators': [10, 30, 50, 70, 90, 110, 130, 150, 170, 190, 210, 230, 250, 270, 290, 310, 330, 350, 370, 390, 410, 430, 450, 470, 490], 'max_features': [1, 5, 6, 7, 10, 12, 15, 20], 'max_depth': [1, 6, 11, 16, 21, 26, 31, 36, 41, 46, 51, 56, 61, 66, 71, 76, 81, 86, 91, 96, 101, 106, 111, 116], 'min_samples_split': array([ 2,  5,  8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38, 41, 44, 47]), 'min_samples_leaf': [1, 6, 11, 16, 21, 26, 31, 36]},
          pre_d

In [5]:
# Utility function to report best scores
def report(results, n_top=10):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


report(rf_random.cv_results_)

Model with rank: 1
Mean validation score: -0.001 (std: 0.000)
Parameters: {'n_estimators': 490, 'min_samples_split': 14, 'min_samples_leaf': 11, 'max_features': 20, 'max_depth': 101}

Model with rank: 2
Mean validation score: -0.001 (std: 0.000)
Parameters: {'n_estimators': 250, 'min_samples_split': 14, 'min_samples_leaf': 16, 'max_features': 20, 'max_depth': 111}

Model with rank: 3
Mean validation score: -0.001 (std: 0.000)
Parameters: {'n_estimators': 470, 'min_samples_split': 23, 'min_samples_leaf': 11, 'max_features': 20, 'max_depth': 96}

Model with rank: 4
Mean validation score: -0.001 (std: 0.000)
Parameters: {'n_estimators': 150, 'min_samples_split': 23, 'min_samples_leaf': 11, 'max_features': 20, 'max_depth': 66}

Model with rank: 5
Mean validation score: -0.001 (std: 0.000)
Parameters: {'n_estimators': 290, 'min_samples_split': 23, 'min_samples_leaf': 11, 'max_features': 20, 'max_depth': 41}

Model with rank: 6
Mean validation score: -0.001 (std: 0.000)
Parameters: {'n_estim

In [13]:
# Simplement aplicarem el mètode amb els paràmetres òptims obtinguts de l'apartat anterior
modelRFOpt = RandomForestRegressor(n_estimators= 490, min_samples_split= 14, min_samples_leaf= 11, 
                              max_features = 20, max_depth= 101)
modelRFOpt.fit(train_X, train_Y)

# predicció amb dades de test
testPredict = modelRFOpt.predict(test_X)



In [24]:
modelRFOpt2 = RandomForestRegressor(n_estimators= 170, min_weight_fraction_leaf= 0, min_samples_split= 67, min_samples_leaf= 10, 
                                   max_leaf_nodes= 54, max_features= None, max_depth= 71, bootstrap = True)
modelRFOpt2.fit(train_X, train_Y)
testPredict2 = modelRFOpt2.predict(test_X)

In [45]:
modelRFOpt3 = RandomForestRegressor(n_estimators= 70, min_weight_fraction_leaf= 0, min_samples_split= 77, min_samples_leaf= 25,
                                    max_leaf_nodes= 52, max_features= 'auto', max_depth= 11, bootstrap= True)
modelRFOpt3.fit(train_X, train_Y)
testPredict3 = modelRFOpt3.predict(test_X)

In [14]:
test_YR = scaler.inverse_transform(test_Y)
testPredict = scaler.inverse_transform(testPredict)

def evaluate_forecasts(actual, predicted):
    scores = list()
    # calculem RMSE per cada dia
    for i in range(actual.shape[1]):
        # calculem MSE
        mse = mean_squared_error(actual[:, i], predicted[:, i])
        # calculem RMSE
        rmse = sqrt(mse)
        # store
        scores.append(rmse)
    # calcul global de RMSE
    s = 0
    for row in range(actual.shape[0]):
        for col in range(actual.shape[1]):
            s += (actual[row, col] - predicted[row, col])**2
    score = sqrt(s / (actual.shape[0] * actual.shape[1]))
    return score, scores

RMSE_TOT, RMSE_days = evaluate_forecasts(test_YR,testPredict)
print(RMSE_TOT)

2.9581023402140523


In [51]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 

# Number of trees in random forest
n_estimators = np.arange(50,90,2).tolist()
# Number of features to consider at every split
max_features = ['auto']
# Maximum number of levels in tree
max_depth = np.arange(5,25,1).tolist()
# Minimum number of samples required to split a node
min_samples_split = [77] #np.arange(70,85,2)
# Minimum number of samples required at each leaf node
min_samples_leaf = [25] #np.arange(20,31,4).tolist()
# Method of selecting samples for training each tree
bootstrap = [True]
max_leaf_nodes = [52]#np.arange(45,60,1).tolist()

param_grid = {
    'bootstrap': bootstrap,
    'max_depth': max_depth,
    'max_features': max_features,
    'min_samples_leaf': min_samples_leaf,
    'min_samples_split': min_samples_split,
    'n_estimators': n_estimators,
    'max_leaf_nodes': max_leaf_nodes
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv=TimeSeriesSplit(n_splits=5).get_n_splits([train_X,train_Y]), scoring='neg_mean_squared_error', n_jobs = -1, verbose = 0)

grid_search.fit(train_X, train_Y)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'bootstrap': [True], 'max_depth': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24], 'max_features': ['auto'], 'min_samples_leaf': [25], 'min_samples_split': [77], 'n_estimators': [50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88], 'max_leaf_nodes': [52]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [55]:
# Utility function to report best scores
def grid_scores_to_df(grid_scores):
    """
    Convert a sklearn.grid_search.GridSearchCV.grid_scores_ attribute to a tidy
    pandas DataFrame where each row is a hyperparameter-fold combinatination.
    """
    rows = list()
    for grid_score in grid_scores:
        for fold, score in enumerate(grid_score.cv_validation_scores):
            row = grid_score.parameters.copy()
            row['fold'] = fold
            row['score'] = score
            rows.append(row)
    df = pd.DataFrame(rows)
    return df

#grid_scores_to_df(grid_search.grid_scores_)
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 5,
 'max_features': 'auto',
 'max_leaf_nodes': 52,
 'min_samples_leaf': 25,
 'min_samples_split': 77,
 'n_estimators': 86}

In [56]:
modelRFOpt4 = RandomForestRegressor(n_estimators= 86, min_weight_fraction_leaf= 0, min_samples_split= 77, min_samples_leaf= 25, 
                                   max_leaf_nodes= 52, max_features= 'auto', max_depth=5, bootstrap = True)
modelRFOpt4.fit(train_X, train_Y)
testPredict4 = modelRFOpt4.predict(test_X)

In [57]:
test_YR4 = scaler.inverse_transform(test_Y)
testPredict4 = scaler.inverse_transform(testPredict4)

def evaluate_forecasts(actual, predicted):
    scores = list()
    # calculem RMSE per cada dia
    for i in range(actual.shape[1]):
        # calculem MSE
        mse = mean_squared_error(actual[:, i], predicted[:, i])
        # calculem RMSE
        rmse = sqrt(mse)
        # store
        scores.append(rmse)
    # calcul global de RMSE
    s = 0
    for row in range(actual.shape[0]):
        for col in range(actual.shape[1]):
            s += (actual[row, col] - predicted[row, col])**2
    score = sqrt(s / (actual.shape[0] * actual.shape[1]))
    return score, scores

RMSE_TOT, RMSE_days = evaluate_forecasts(test_YR4,testPredict4)
print(RMSE_TOT)

3.23505358607499


In [58]:
from tabulate import tabulate
headers=[]
for i in range(len(RMSE_days)):
    headers.append('dia '+str(i+1))

headers.append('Total')
v = RMSE_days
ultim = v.append(RMSE_TOT)
table1 = tabulate([RMSE_days[0:5]], headers[0:5], tablefmt="fancy_grid")
table2 = tabulate([RMSE_days[5:10]], headers[5:10], tablefmt="fancy_grid")
table3 = tabulate([RMSE_days[10:16]], headers[10:16], tablefmt="fancy_grid")
#output
print(table1)
print(table2)
print(table3)

╒══════════╤═════════╤═════════╤═════════╤═════════╕
│    dia 1 │   dia 2 │   dia 3 │   dia 4 │   dia 5 │
╞══════════╪═════════╪═════════╪═════════╪═════════╡
│ 0.899697 │ 1.21273 │  1.5334 │  1.8448 │ 2.14941 │
╘══════════╧═════════╧═════════╧═════════╧═════════╛
╒═════════╤═════════╤═════════╤═════════╤══════════╕
│   dia 6 │   dia 7 │   dia 8 │   dia 9 │   dia 10 │
╞═════════╪═════════╪═════════╪═════════╪══════════╡
│ 2.44922 │ 2.74741 │ 3.04331 │ 3.33897 │  3.62501 │
╘═════════╧═════════╧═════════╧═════════╧══════════╛
╒══════════╤══════════╤══════════╤══════════╤══════════╤═════════╕
│   dia 11 │   dia 12 │   dia 13 │   dia 14 │   dia 15 │   Total │
╞══════════╪══════════╪══════════╪══════════╪══════════╪═════════╡
│  3.89586 │  4.15215 │  4.40075 │   4.6455 │  4.88523 │ 3.23505 │
╘══════════╧══════════╧══════════╧══════════╧══════════╧═════════╛
